In [2]:
# Spark libs
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.window import Window
import pyspark.sql.functions as func
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql.functions import monotonically_increasing_id
from pyspark import SQLContext
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.functions import lit

# Others
import pandas as pd
import numpy as np
import datetime
import time

# Graphs libs
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt_dt

# S3 Service
# import boto3
# from io import StringIO

# # Let's use Amazon S3
# s3 = boto3.resource('s3')


spark.version

NameError: name 'spark' is not defined

In [3]:
OBV_i = True
CCI_i = True
RSI_i = True
MACD_i = True
ManualSplit = False
Date_Convert = False

DEBUG = True

DT = True
RT = False
GBT = False

RANDOM_SEED = 1

In [4]:
def RSI(dataframe, window_length, avg_type, column='Close'):
    data = dataframe.toPandas()
    # Get just the close
    close = data['Close']
    # Get the difference in price from previous step
    delta = close.diff()
    # Get rid of the first row, which is NaN since it did not have a previous
    # row to calculate the differences
    # Make the positive gains (up) and negative gains (down) Series
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    if avg_type == "EWMA":
        roll_up = up.ewm(span=window_length, min_periods=window_length).mean()
        roll_down = down.abs().ewm(
            span=window_length, min_periods=window_length).mean()
    elif avg_type == "SMA":
        roll_up = pd.rolling_mean(up, window_length)
        roll_down = pd.rolling_mean(down.abs(), window_length)
    RS = roll_up / roll_down
    RSI = 100.0 - (100.0 / (1.0 + RS))
    RSI = pd.DataFrame({'RSI': RSI})
    data = data.join(RSI)
    result_df = spark.createDataFrame(data.round(3))
    return result_df

In [5]:
# Commodity Channel Index
def CCI(spark_df, ndays):
    data = spark_df.toPandas()
    TP = (data['High'] + data['Low'] + data['Close']) / 3
    CCI = pd.Series(
        (TP - pd.rolling_mean(TP, ndays)) /
        (0.015 * pd.rolling_std(TP, ndays)),
        name='CCI')
    data = data.join(CCI)
    result_df = spark.createDataFrame(data.round(3))
    return result_df

In [6]:
# Moving average convergence divergence
def MACD(dataframe, nfast=12, nslow=26, signal=9, column='Close'):
    data = dataframe.toPandas()
    # Get just the close
    price = data[column]
    # Get the difference in price from previous step
    emaslow = pd.ewma(price, span=nslow, min_periods=1)
    emafast = pd.ewma(price, span=nfast, min_periods=1)
    #     MACD = pd.DataFrame({'MACD': emafast-emaslow, 'emaSlw': emaslow, 'emaFst': emafast})
    MACD = pd.DataFrame({'MACD': emafast - emaslow})

    data = data.join(MACD.round(3))
    result_df = spark.createDataFrame(data)
    return result_df

In [7]:
# UDF FUNCTIONS
def profit_(today_price, previous_day_price):
    if today_price - previous_day_price > 0:
        return 1
    else:
        return 0


profit_udf = udf(profit_, IntegerType())

In [8]:
# Spark context simple configuration
spark = SparkSession.builder.config(conf=SparkConf()).getOrCreate()

#Path to file with CSV

# path_to_csv = "s3://stocksets100/Orlen.csv"
path_to_csv = "./Datasets/Orlen.csv"

fresh_df = spark.read.csv(path_to_csv, header=True, inferSchema=True)

fresh_df = fresh_df.filter(fresh_df.Open != "null")

temporary_df = fresh_df.select(
    fresh_df["Date"].cast("Date"), fresh_df["Open"].cast("float"),
    fresh_df["High"].cast("float"), fresh_df["Volume"].cast("int"),
    fresh_df["Low"].cast("float"), fresh_df["Close"].cast("float"))

# Creating new column with shifted Close price by 1 day
df_daily_return = temporary_df.withColumn('prev_day_price',
                                 func.lag(temporary_df['Close']).over(
                                     Window.orderBy("Date")))

df_daily_return = df_daily_return.filter(
    df_daily_return.prev_day_price.isNotNull())

# Profit label calculation
# 1 if stock risen up, 0 is it went down
df_profit = df_daily_return.withColumn(
    'Profit', profit_udf(df_daily_return.Close,
                         df_daily_return.prev_day_price))

df_shifted_profit = df_profit.withColumn(
    'Profit',
    func.lag(df_profit['Profit'], count=-1).over(Window.orderBy("Date")))

final_df = df_shifted_profit.filter(df_shifted_profit.Profit.isNotNull())

# Removing redudant columns
final_df = final_df.drop("Daily return")
final_df = final_df.drop("prev_day_price")

#Conversion to desired typesf
converted_df = final_df.select("*").withColumn("id",
                                               monotonically_increasing_id())

converted_df = converted_df.select("id", 'Date', 'Open', 'High', 'Volume',
                                   'Low', 'Close', 'Profit')
if DEBUG:
    converted_df.show()

+---+----------+-----+-----+------+-----+-----+------+
| id|      Date| Open| High|Volume|  Low|Close|Profit|
+---+----------+-----+-----+------+-----+-----+------+
|  0|2012-05-29|7.802|7.809|     0|7.802|7.809|     0|
|  1|2012-05-30| 7.95| 7.95|    10|7.782|7.782|     0|
|  2|2012-05-31|7.688|7.688|     0|7.688|7.688|     0|
|  3|2012-06-01|7.526|7.526|   300|7.459|7.459|     0|
|  4|2012-06-04|7.328|7.328|     0|7.328|7.328|     1|
|  5|2012-06-05|7.514|7.514|     0|7.514|7.514|     0|
|  6|2012-06-06|7.317|7.317|     0|7.317|7.317|     1|
|  7|2012-06-08|7.502|7.569|     0|7.502|7.569|     1|
|  8|2012-06-11|7.701|7.701|     0|7.701|7.701|     0|
|  9|2012-06-12|7.671|7.671|     0|7.671|7.671|     1|
| 10|2012-06-13|7.935|7.935|     0|7.935|7.935|     1|
| 11|2012-06-14|7.978|7.978|     0|7.978|7.978|     1|
| 12|2012-06-15|8.089|8.089|     0|8.089|8.089|     1|
| 13|2012-06-18|8.337|8.337|     0|8.337|8.337|     0|
| 14|2012-06-19|8.223|8.223|     0|8.223|8.223|     1|
| 15|2012-

In [9]:
#Columns without Date
# converted_df = converted_df.select(
#     [col(c).cast('float') for c in converted_df.columns if c. not in {'Date'}])
# Date column!
df_date = converted_df.select(converted_df.Date)
df_date = df_date.select("*").withColumn("id", monotonically_increasing_id())

#Convert date to splitted format
if Date_Convert:
    split_col = pyspark.sql.functions.split(converted_df['Date'], '-')
    converted_df = converted_df.withColumn('Year',
                                           split_col.getItem(0).cast('int'))
    converted_df = converted_df.withColumn('Month',
                                           split_col.getItem(1).cast('int'))
    converted_df = converted_df.withColumn('Day',
                                           split_col.getItem(2).cast('int'))
    if DEBUG:
        converted_df.show()

converted_df = converted_df.drop("Date")

In [10]:
if MACD_i:
    converted_df = MACD(converted_df)
    converted_df = converted_df.select(
        [col(c).cast('float') for c in converted_df.columns])
    if DEBUG:
        converted_df.show()

/home/zapo/Documents/Spark/spark/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(span=26,min_periods=1,adjust=True,ignore_na=False).mean()
  import sys
/home/zapo/Documents/Spark/spark/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: pd.ewm_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.ewm(span=12,min_periods=1,adjust=True,ignore_na=False).mean()
  


+----+-----+-----+------+-----+-----+------+------+
|  id| Open| High|Volume|  Low|Close|Profit|  MACD|
+----+-----+-----+------+-----+-----+------+------+
| 0.0|7.802|7.809|   0.0|7.802|7.809|   0.0|   0.0|
| 1.0| 7.95| 7.95|  10.0|7.782|7.782|   0.0|-0.001|
| 2.0|7.688|7.688|   0.0|7.688|7.688|   0.0|-0.004|
| 3.0|7.526|7.526| 300.0|7.459|7.459|   0.0|-0.013|
| 4.0|7.328|7.328|   0.0|7.328|7.328|   1.0|-0.023|
| 5.0|7.514|7.514|   0.0|7.514|7.514|   0.0|-0.021|
| 6.0|7.317|7.317|   0.0|7.317|7.317|   1.0|-0.028|
| 7.0|7.502|7.569|   0.0|7.502|7.569|   1.0|-0.021|
| 8.0|7.701|7.701|   0.0|7.701|7.701|   0.0|-0.008|
| 9.0|7.671|7.671|   0.0|7.671|7.671|   1.0|-0.001|
|10.0|7.935|7.935|   0.0|7.935|7.935|   1.0| 0.018|
|11.0|7.978|7.978|   0.0|7.978|7.978|   1.0| 0.034|
|12.0|8.089|8.089|   0.0|8.089|8.089|   1.0| 0.052|
|13.0|8.337|8.337|   0.0|8.337|8.337|   0.0| 0.079|
|14.0|8.223|8.223|   0.0|8.223|8.223|   1.0| 0.092|
|15.0|8.587|8.824| 370.0|8.587|8.824|   0.0| 0.137|
|16.0|8.634|

In [11]:
if CCI_i:
    converted_df = CCI(converted_df, 14)
    converted_df = converted_df.select(
        [col(c).cast('float') for c in converted_df.columns])
    converted_df = converted_df.filter(converted_df.CCI != "NaN")
    if DEBUG:
        converted_df.show()

/home/zapo/Documents/Spark/spark/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=14,center=False).mean()
  
/home/zapo/Documents/Spark/spark/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=14,center=False).std()
  import sys


+----+-----+-----+------+-----+-----+------+-----+-------+
|  id| Open| High|Volume|  Low|Close|Profit| MACD|    CCI|
+----+-----+-----+------+-----+-----+------+-----+-------+
|13.0|8.337|8.337|   0.0|8.337|8.337|   0.0|0.079|139.019|
|14.0|8.223|8.223|   0.0|8.223|8.223|   1.0|0.092| 96.656|
|15.0|8.587|8.824| 370.0|8.587|8.824|   0.0|0.137|148.128|
|16.0|8.634|8.634| 350.0|8.616|8.616|   0.0|0.157|105.222|
|17.0|8.404|8.457|1250.0|8.404|8.441|   0.0|0.159| 67.586|
|18.0|8.402|8.402|   0.0|8.402|8.402|   0.0|0.156| 54.927|
|19.0|8.344|8.344|   0.0|8.344|8.344|   1.0|0.148| 39.074|
|20.0|8.583|8.583|   0.0|8.583|8.583|   0.0|0.156| 70.221|
|21.0|8.471|8.471|   0.0|8.471|8.471|   1.0|0.153| 43.572|
|22.0|8.564|8.564|   0.0|8.564|8.564|   1.0|0.155| 54.995|
|23.0|8.828|8.828|   0.0|8.828|8.828|   1.0|0.173| 106.49|
|24.0|8.929|8.929|   0.0|8.929|8.929|   0.0|0.192|113.932|
|25.0|8.902|8.902|   0.0|8.902|8.902|   1.0|0.202| 96.888|
|26.0|9.092|9.092|   0.0|9.092|9.092|   0.0|0.222|124.93

In [12]:
# OBV indicator calculation
if OBV_i:
    temp_df = converted_df.toPandas()
    df_obv = spark.createDataFrame(
        temp_df.assign(OBV=(temp_df.Volume * (
            ~temp_df.Close.diff().le(0) * 2 - 1)).cumsum()))
    converted_df = df_obv.select(
        [col(c).cast('float') for c in df_obv.columns])
    if DEBUG:
        converted_df.show()

+----+-----+-----+------+-----+-----+------+-----+-------+-------+
|  id| Open| High|Volume|  Low|Close|Profit| MACD|    CCI|    OBV|
+----+-----+-----+------+-----+-----+------+-----+-------+-------+
|13.0|8.337|8.337|   0.0|8.337|8.337|   0.0|0.079|139.019|    0.0|
|14.0|8.223|8.223|   0.0|8.223|8.223|   1.0|0.092| 96.656|    0.0|
|15.0|8.587|8.824| 370.0|8.587|8.824|   0.0|0.137|148.128|  370.0|
|16.0|8.634|8.634| 350.0|8.616|8.616|   0.0|0.157|105.222|   20.0|
|17.0|8.404|8.457|1250.0|8.404|8.441|   0.0|0.159| 67.586|-1230.0|
|18.0|8.402|8.402|   0.0|8.402|8.402|   0.0|0.156| 54.927|-1230.0|
|19.0|8.344|8.344|   0.0|8.344|8.344|   1.0|0.148| 39.074|-1230.0|
|20.0|8.583|8.583|   0.0|8.583|8.583|   0.0|0.156| 70.221|-1230.0|
|21.0|8.471|8.471|   0.0|8.471|8.471|   1.0|0.153| 43.572|-1230.0|
|22.0|8.564|8.564|   0.0|8.564|8.564|   1.0|0.155| 54.995|-1230.0|
|23.0|8.828|8.828|   0.0|8.828|8.828|   1.0|0.173| 106.49|-1230.0|
|24.0|8.929|8.929|   0.0|8.929|8.929|   0.0|0.192|113.932|-123

In [13]:
#RSI indicator calculaction
if RSI_i:
    converted_df = RSI(converted_df, 3, 'SMA')
    converted_df = converted_df.filter(converted_df.RSI != "NaN")
    if DEBUG:
        converted_df.show()

/home/zapo/Documents/Spark/spark/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=3,center=False).mean()
/home/zapo/Documents/Spark/spark/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=3,center=False).mean()


+----+-----+-----+------+-----+-----+------+-----+-------+-------+------+
|  id| Open| High|Volume|  Low|Close|Profit| MACD|    CCI|    OBV|   RSI|
+----+-----+-----+------+-----+-----+------+-----+-------+-------+------+
|16.0|8.634|8.634| 350.0|8.616|8.616|   0.0|0.157|105.222|   20.0|65.114|
|17.0|8.404|8.457|1250.0|8.404|8.441|   0.0|0.159| 67.586|-1230.0|61.077|
|18.0|8.402|8.402|   0.0|8.402|8.402|   0.0|0.156| 54.927|-1230.0|   0.0|
|19.0|8.344|8.344|   0.0|8.344|8.344|   1.0|0.148| 39.074|-1230.0|   0.0|
|20.0|8.583|8.583|   0.0|8.583|8.583|   0.0|0.156| 70.221|-1230.0|71.131|
|21.0|8.471|8.471|   0.0|8.471|8.471|   1.0|0.153| 43.572|-1230.0|58.435|
|22.0|8.564|8.564|   0.0|8.564|8.564|   1.0|0.155| 54.995|-1230.0|74.775|
|23.0|8.828|8.828|   0.0|8.828|8.828|   1.0|0.173| 106.49|-1230.0|76.119|
|24.0|8.929|8.929|   0.0|8.929|8.929|   0.0|0.192|113.932|-1230.0| 100.0|
|25.0|8.902|8.902|   0.0|8.902|8.902|   1.0|0.202| 96.888|-1230.0|93.112|
|26.0|9.092|9.092|   0.0|9.092|9.092| 

In [14]:
#   df_date = df_date.toPandas()

In [16]:
# if DEBUG:
#     df_date = df_date.set_index('id')
#     df_to_plot_dt = converted_df.select([
#         c for c in converted_df.columns
#         if c not in
#         {'OBV', 'Volume', 'Low', 'High', 'Open', 'Profit', 'RSI', 'CCI'}
#     ])
#     df_to_plot_dt = df_to_plot_dt.withColumn('Zero',lit(0))
#     df_to_plot_dt = df_to_plot_dt.toPandas()
#     df_to_plot_dt = df_to_plot_dt.set_index('id')
#     df_to_plot_dt = df_to_plot_dt.join(df_date)
#     plt_dt.figure(figsize=(24, 10))
#     plt_dt.plot(df_to_plot_dt.Date, df_to_plot_dt.Close,df_to_plot_dt.Zero )
#     plt_dt.legend(df_to_plot_dt.columns)
#     plt_dt.show()

KeyError: 'id'

In [17]:
converted_df = converted_df.sort(converted_df.id.asc())

# Manual split for training and validating data
if ManualSplit:
    dfp = converted_df.toPandas()

    dfp = np.array_split(dfp, 10)

    p0 = spark.createDataFrame(data=dfp[0])
    p1 = spark.createDataFrame(data=dfp[1])
    p2 = spark.createDataFrame(data=dfp[2])
    p3 = spark.createDataFrame(data=dfp[3])
    p4 = spark.createDataFrame(data=dfp[4])
    p5 = spark.createDataFrame(data=dfp[5])
    p6 = spark.createDataFrame(data=dfp[6])
    p7 = spark.createDataFrame(data=dfp[7])
    p8 = spark.createDataFrame(data=dfp[8])
    p9 = spark.createDataFrame(data=dfp[9])

    p_final = p0.union(p1).union(p2).union(p3).union(p4).union(p5).union(
        p6).union(p7).union(p8)
    train = p_final
    test = p9
    #     test = p9.head(10)
    #     test = spark.createDataFrame(test)
else:
    train, test = converted_df.randomSplit([0.9, 0.1], seed=RANDOM_SEED)

print("We have %d training examples and %d test examples." % (train.count(),
                                                              test.count()))
test = test.select([col(c).cast('float') for c in test.columns])

train = train.select([col(c).cast('float') for c in train.columns])

test = test.sort(test.id.asc())

train = train.sort(train.id.asc())

if DEBUG:
    train.show()

+----+-----+-----+------+-----+-----+------+-----+-------+-------+------+
|  id| Open| High|Volume|  Low|Close|Profit| MACD|    CCI|    OBV|   RSI|
+----+-----+-----+------+-----+-----+------+-----+-------+-------+------+
|16.0|8.634|8.634| 350.0|8.616|8.616|   0.0|0.157|105.222|   20.0|65.114|
|17.0|8.404|8.457|1250.0|8.404|8.441|   0.0|0.159| 67.586|-1230.0|61.077|
|18.0|8.402|8.402|   0.0|8.402|8.402|   0.0|0.156| 54.927|-1230.0|   0.0|
|19.0|8.344|8.344|   0.0|8.344|8.344|   1.0|0.148| 39.074|-1230.0|   0.0|
|20.0|8.583|8.583|   0.0|8.583|8.583|   0.0|0.156| 70.221|-1230.0|71.131|
|21.0|8.471|8.471|   0.0|8.471|8.471|   1.0|0.153| 43.572|-1230.0|58.435|
|22.0|8.564|8.564|   0.0|8.564|8.564|   1.0|0.155| 54.995|-1230.0|74.775|
|23.0|8.828|8.828|   0.0|8.828|8.828|   1.0|0.173| 106.49|-1230.0|76.119|
|24.0|8.929|8.929|   0.0|8.929|8.929|   0.0|0.192|113.932|-1230.0| 100.0|
|25.0|8.902|8.902|   0.0|8.902|8.902|   1.0|0.202| 96.888|-1230.0|93.112|
|26.0|9.092|9.092|   0.0|9.092|9.092| 

In [ ]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler, VectorIndexer

featuresCols = converted_df.columns
featuresCols.remove('Profit')
featuresCols.remove('id')

print(featuresCols)

# Vector Assembler
# This concatenates all feature columns into a single feature vector in a new column "rawFeatures".
# Used for assembling features into a vector.
# We will pass all the columns that we are going to use for the prediction to the VectorAssembler and
# it will create a new vector column.
vectorAssembler_rt = VectorAssembler(
    inputCols=featuresCols, outputCol="rawFeatures")

# VectorIndexer:
# is used to index categorical predictors in a featuresCol column.
# Remember that featuresCol is a single column consisting of vectors (refer to featuresCol and labelCol).
# Each row is a vector which contains values from each predictors.

featureIndexer_rt = VectorIndexer(
    inputCol="rawFeatures",
    outputCol="features",
    maxCategories=len(featuresCols))

In [ ]:
# Algorithm
from pyspark.ml.classification import DecisionTreeClassifier

rt = DecisionTreeClassifier(
    labelCol='Profit', featuresCol="features", minInfoGain=0.01)
max_Depth_Range = list(range(5, 20))
min_InstancesPerNode = list(range(1, 5))

paramGrid_rt = ParamGridBuilder() \
    .addGrid(rt.maxDepth, max_Depth_Range) \
    .addGrid(rt.maxMemoryInMB, [1000] ).build()

# We define an evaluation metric. This tells CrossValidator how well we are doing by comparing the true
# labels with predictions.

evaluator_rt = BinaryClassificationEvaluator(
    labelCol=rt.getLabelCol(),
    metricName='areaUnderROC',
    rawPredictionCol=rt.getRawPredictionCol())

evaluator_rt_PR = BinaryClassificationEvaluator(
    labelCol=rt.getLabelCol(),
    metricName='areaUnderPR',
    rawPredictionCol=rt.getRawPredictionCol())

# Declare the CrossValidator, which runs model tuning for us.
cv_rt = CrossValidator(
    estimator=rt,
    evaluator=evaluator_rt,
    estimatorParamMaps=paramGrid_rt,
    numFolds=10)

In [ ]:
from pyspark.ml.classification import DecisionTreeClassificationModel

DecisionTreeClassificationModel.getMaxDepth = (
    lambda self: self._java_obj.getMaxDepth())

DecisionTreeClassificationModel.getMinInstancesPerNode = (
    lambda self: self._java_obj.getMinInstancesPerNode())

In [ ]:
# if DEBUG:
#     train.repartition(1).write.csv("PreProcessedSets/TrainSet_" + str(time.mktime(datetime.datetime.today().timetuple())) + "_.csv", header = 'True')
#     test.repartition(1).write.csv("PreProcessedSets/TestSet_" + str(time.mktime(datetime.datetime.today().timetuple())) + "_.csv", header = 'True')

In [ ]:
# Creating Final pipeline object
pipeline_rt = Pipeline(stages=[vectorAssembler_rt, featureIndexer_rt, cv_rt])

# FITTING!
pipelineModel_rt = pipeline_rt.fit(train)

# Getting the Best Model
best_classifier = pipelineModel_rt.stages[-1].bestModel

In [ ]:
# Feature importance
print('Features importances' + str(best_classifier.featureImportances))

# Max depth
print("Maximal depth is " + str(best_classifier.getMaxDepth()))
max_depth = best_classifier.getMaxDepth()

# Min instances
print("Minimal instances per node is " + str(
    best_classifier.getMinInstancesPerNode()))
min_instancesPerNode = best_classifier.getMinInstancesPerNode()

# Making Predictions!
predictions_rt = pipelineModel_rt.transform(test)

# Calculating metrics
AreaUnderROC = evaluator_rt.evaluate(predictions_rt)
print("AreaUnderROC on our test set: %g" % AreaUnderROC)

# Calculating metrics
AreaUnderPR = evaluator_rt_PR.evaluate(predictions_rt)
print("AreaUnderPR on our test set: %g" % AreaUnderPR)

#evaluate results
testCount = predictions_rt.count()

FP = predictions_rt.where("prediction = 0 AND Profit=1").count()  #FN
FN = predictions_rt.where("prediction = 1 AND Profit=0").count()  #TN
TP = predictions_rt.where("prediction = 1 AND Profit=1").count()  #TP
TN = predictions_rt.where("prediction = 0 AND Profit=0").count()  #TN

print("Test count | FN | TN | TP | TN")
print(
    str(testCount) + " | " + str(FP) + " | " + str(FN) + " | " + str(TP) +
    " | " + str(TN))

# predictions_rt.show()

In [ ]:
# final_model = pipelineModel_rt
from random import *

for i in range(10):
    new_train, new_test = converted_df.randomSplit([0.9, 0.1], seed=i + 1)
    new_test = new_test.sort(new_test.id.asc())

    predictions = pipelineModel_rt.transform(new_test)

    # Calculating metrics
    AreaUnderROC = evaluator_rt.evaluate(predictions)
    print("AreaUnderROC on our test set: %g" % AreaUnderROC)

    # Calculating metrics
    AreaUnderPR = evaluator_rt_PR.evaluate(predictions)
    print("AreaUnderPR on our test set: %g" % AreaUnderPR)

    #evaluate results
    testCount = predictions_rt.count()

    FP = predictions_rt.where("prediction = 0 AND Profit=1").count()  #FN
    FN = predictions_rt.where("prediction = 1 AND Profit=0").count()  #TN
    TP = predictions_rt.where("prediction = 1 AND Profit=1").count()  #TP
    TN = predictions_rt.where("prediction = 0 AND Profit=0").count()  #TN

    print("Test count | FN | TN | TP | TN")
    print(
        str(testCount) + " | " + str(FP) + " | " + str(FN) + " | " + str(TP) +
        " | " + str(TN))

    print("####################################################\n\n")

In [ ]:
test = predictions_rt.toPandas()
csv_buffer = StringIO()
test.to_csv(csv_buffer)

s3_resource = boto3.resource('s3')

s3_resource.Object('logs102', 'DT_Final.csv').put(Body=csv_buffer.getvalue())

if DEBUG != True:
    df_to_plot_rt = predictions_rt.select('prediction', 'Profit')
    print(df_to_plot_rt)
    df_to_plot_rt = df_to_plot_rt.toPandas()
    plt_dt.figure(figsize=(14, 14))
    plt_dt.plot(df_to_plot_rt)
    plt_dt.legend(df_to_plot_rt.columns)
    plt_dt.show()